# **Finding Lane Lines in Video** 
***
This is a project using the teaser info that I learned before the actual course was released.
In this notebook I am working on the test clip to try extracting the lanes. This clip is specifically more dificult than wat I've seen until now in the course because:
- I have to read in a movie clip, the course showed only images  
- The lanes in this clip are specifically bendy, meaning the car is not driving in a strait line

Never the less this small project uses all the functions learn for image editing. The video that will be used is 'test.mp4'

In [1]:
# Importing relevant packages for data manipulation
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
%matplotlib inline

In [2]:
# Importing relevant packages for computer vision
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [3]:
# Creating editing functions
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img, vertices):
    # initializing a blank mask
    mask = np.zeros_like(img)
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 13], thickness=8):
    for line in lines:
        for x1,y1,x2,y2 in line:
            if y2!=y1:
                slope = float(x2-x1)/(y2-y1)
                y_t = 0
                if slope < 0: # left lane, negative slope
                    xMIN = slope*(img.shape[0]-1-y1) + x1
                    xMAX = slope*(330-y1) + x1
                    y_t=330
                elif slope > 0: # right lane, positive slope
                    xMIN = slope*(img.shape[0]-1-y1) + x1
                    xMAX = slope*(360-y1) + x1
                    y_t = 360
                cv2.line(img, (int(xMIN),img.shape[0]-1),(int(xMAX),y_t), color, thickness)
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img
    
def weighted_img(init_img, line_img, alpha=0.8, beta=1.0, gamma=0.0):
    return cv2.addWeighted(init_img, alpha, line_img, beta, gamma)

def draw_lines2(img, lines, color=[255, 0, 0], thickness=3):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines2(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines2(line_img, lines)
    return line_img


In [4]:
# Creating a pipeline for frame editing
def process_video(img):
    gray = grayscale(img)
    
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)
    
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    imgsh = img.shape
    vertices = np.array([[(200,imgsh[0]-55),(660,430),(675,430),(imgsh[1]-175,imgsh[0]-55)]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10 # minimum number of votes (intersections in Hough grid cell)
    min_len = 10
    max_gap = 20
    line_img = hough_lines2(masked_edges, rho, theta, threshold, min_len, max_gap)
    
    result = weighted_img(img, line_img, 0.8, 1.2, 0)
    
    return result

In [5]:
# Applying the pipeline on the clip, frame by frame
file_output = 'curvy_out.mp4'
clip1 = VideoFileClip('curvy.mp4')
file_clip = clip1.fl_image(process_video)
%time file_clip.write_videofile(file_output, audio=False)

[MoviePy] >>>> Building video curvy_out.mp4
[MoviePy] Writing video curvy_out.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:10<00:00, 23.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: curvy_out.mp4 

Wall time: 12 s


In [6]:
#cv2.addWeighted?